# LAB7 Assignment
> The document description are designed by JIa Yanhong in 2022. Oct. 20th
------

## LAB Assignment
### Exercise 1 logistic regression
This exercise uses dataset digit01.csv , which has 13 columns, and the last column is the dependent variable. 

This part requires you to implement a `logistic regression` using the pytorch framework (defining a logistic regression class that inherits `nn.module`). To test your model, we provide a dataset `digit01.csv` which is in the **datasets folder**. This dataset requires you to divide the training set and the test set by yourself, and it is recommended that 80% of the training set and 20% of the test set be used.

+ load datasets

In [5]:
########### Write Your Code Here ###########
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline

# to suppress warnings
import warnings
warnings.filterwarnings("ignore") 
df = pd.read_csv("datasets/digit01.csv", header=None)
df.head()
############################################

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,1,1,1,0,1,1,0,1,1,1,1,0
1,0,1,1,1,0,1,1,0,1,1,1,1,0
2,1,1,0,1,0,1,1,0,1,1,1,1,0
3,1,1,1,1,0,1,1,0,1,1,1,0,0
4,1,1,1,1,0,1,1,0,1,0,1,1,0


+ Splitting dataset into 80% Training and 20% Testing Data:

In [8]:
########### Write Your Code Here ###########
y = df[12].values
X = df.drop(columns=12).values
print(X.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state =0)
############################################


(64, 12)


+ Define a LogisticRegression subclass of nn. Module

In [9]:
# Define a LogisticRegression subclass of nn. Module.
########### Write Your Code Here ###########
import torch
from torch import nn

X_train_tensor = torch.FloatTensor(X_train)
X_test_tensor = torch.FloatTensor(X_test)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)
y_test_tensor = torch.FloatTensor(y_test).view(-1, 1)

class LogisticRegression(nn.Module):
    def __init__(self, input_size):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))
############################################


+ Create the model

In [10]:
########### Write Your Code Here ###########
input_size = X_train.shape[1]
model = LogisticRegression(input_size)
############################################


 + Loss function

In [16]:
########### Write Your Code Here ###########
loss_fn = torch.nn.MSELoss(reduction='sum')
############################################

+ The optimizer

In [17]:
########### Write Your Code Here ###########
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=0.01)
############################################

+ training Model

In [20]:
########### Write Your Code Here ###########
num_epochs = 2000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = loss_fn(outputs, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print loss every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f'{epoch+1} {loss.item():.6f}')
############################################

100 0.093916
200 0.087165
300 0.081661
400 0.076852
500 0.072591
600 0.068783
700 0.065357
800 0.062256
900 0.059435
1000 0.056858
1100 0.054494
1200 0.052317
1300 0.050306
1400 0.048443
1500 0.046711
1600 0.045098
1700 0.043591
1800 0.042181
1900 0.040859
2000 0.039616



+ Model Performance


In [24]:
########### Write Your Code Here ###########
from sklearn.metrics import accuracy_score
with torch.no_grad():
    y_pred_train = model(X_train_tensor).round()
    y_pred_test = model(X_test_tensor).round()
    
    train_accuracy = accuracy_score(y_train_tensor.numpy(), y_pred_train.numpy())
    test_accuracy = accuracy_score(y_test_tensor.numpy(), y_pred_test.numpy())

print(f'Train Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')
weights = model.linear.weight.detach().numpy()[0]  # Convert to numpy array and get the first row
bias = model.linear.bias.detach().numpy()[0]        # Convert to numpy array

# Prepare the output in a specified format
coefficients_str = " + ".join(f"{weight:.6f} x^{i+1}" for i, weight in enumerate(weights))
print(f"Result: y = {bias:.6f} + {coefficients_str}")

############################################

Train Accuracy: 1.0000
Test Accuracy: 1.0000
Result: y = 0.776317 + -0.853705 x^1 + 2.233681 x^2 + -1.178715 x^3 + -1.755139 x^4 + 1.921055 x^5 + -2.255334 x^6 + -1.313849 x^7 + 1.768014 x^8 + -2.011778 x^9 + -1.789865 x^10 + 2.208385 x^11 + -0.760377 x^12


In [25]:
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         7

    accuracy                           1.00        13
   macro avg       1.00      1.00      1.00        13
weighted avg       1.00      1.00      1.00        13



### Exercise 2  Handwriting recognition with MLP

Like last week's lab , your task in this section is also about recognizing handwritten digits, but you are required to use MLP to complete the exercise. It is recommended that you define an MLP class, which is a subclass of `nn.module`.


For this exercise we use the `minist` dataset.

+ load datasets

In [20]:
########### Write Your Code Here ###########
import numpy as np
from torch.utils.data import Dataset, DataLoader

def load_mnist_images(file_path):
    with open(file_path, 'rb') as f:
        f.read(16)
        images = np.frombuffer(f.read(), dtype=np.uint8)
        # 进行reshape，格式为(数量, 高, 宽)
        return images.reshape(-1, 28, 28)

def load_mnist_labels(file_path):
    with open(file_path, 'rb') as f:
        f.read(8)
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels
    
class MNISTDataset(Dataset):
    def __init__(self, images_file, labels_file):
        self.images = load_mnist_images(images_file)
        self.labels = load_mnist_labels(labels_file)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        image = self.images[index].astype(np.float32) / 255.0  # 归一化到 [0, 1]
        label = self.labels[index]
        return torch.tensor(image), torch.tensor(label)

path = "datasets/MNIST/raw/"
train_images_file = path + 'train-images-idx3-ubyte'
train_labels_file = path + 'train-labels-idx1-ubyte'
test_images_file = path + 't10k-images-idx3-ubyte'
test_labels_file = path + 't10k-labels-idx1-ubyte'

train_dataset = MNISTDataset(train_images_file, train_labels_file)
test_dataset = MNISTDataset(test_images_file, test_labels_file)

# 创建数据加载器
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)
 
############################################

+ Define a MLP subclass of nn. Module

In [21]:
########### Write Your Code Here ###########
import torch
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # 第一层
        self.fc2 = nn.Linear(hidden_size, num_classes)  # 输出层
        self.relu = nn.ReLU()  # 激活函数

    def forward(self, x):
        x = x.view(x.size(0), -1)  # 将28x28图像展平
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
############################################

+ Create the model

In [33]:
########### Write Your Code Here ###########
input_size = 28 * 28  # 图像展平后的尺寸
hidden_size = 32 # 隐藏层神经元数量
num_classes = 10  # 0-9

model = MLP(input_size, hidden_size, num_classes)
############################################

 + Loss function

In [34]:
########### Write Your Code Here ###########
criterion = nn.CrossEntropyLoss()
############################################

+ The optimizer

In [35]:
########### Write Your Code Here ###########
import torch.optim as optim
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
############################################

+ training Model

In [50]:

########### Write Your Code Here ###########
num_epochs = 5  # 训练轮数
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(f'Epoch [{epoch + 1}, {i+1}], Loss: {loss.item():.4f}')

############################################

Epoch [1, 1], Loss: 0.0115
Epoch [1, 101], Loss: 0.0086
Epoch [1, 201], Loss: 0.0354
Epoch [1, 301], Loss: 0.0170
Epoch [1, 401], Loss: 0.0503
Epoch [1, 501], Loss: 0.0059
Epoch [1, 601], Loss: 0.0377
Epoch [1, 701], Loss: 0.0574
Epoch [1, 801], Loss: 0.0221
Epoch [1, 901], Loss: 0.0163
Epoch [2, 1], Loss: 0.0299
Epoch [2, 101], Loss: 0.0338
Epoch [2, 201], Loss: 0.0139
Epoch [2, 301], Loss: 0.0225
Epoch [2, 401], Loss: 0.0312
Epoch [2, 501], Loss: 0.0040
Epoch [2, 601], Loss: 0.0288
Epoch [2, 701], Loss: 0.0077
Epoch [2, 801], Loss: 0.0649
Epoch [2, 901], Loss: 0.0094
Epoch [3, 1], Loss: 0.0705
Epoch [3, 101], Loss: 0.0062
Epoch [3, 201], Loss: 0.0244
Epoch [3, 301], Loss: 0.0172
Epoch [3, 401], Loss: 0.0195
Epoch [3, 501], Loss: 0.0441
Epoch [3, 601], Loss: 0.0112
Epoch [3, 701], Loss: 0.0010
Epoch [3, 801], Loss: 0.0186
Epoch [3, 901], Loss: 0.0190
Epoch [4, 1], Loss: 0.0105
Epoch [4, 101], Loss: 0.0059
Epoch [4, 201], Loss: 0.0138
Epoch [4, 301], Loss: 0.0124
Epoch [4, 401], Loss: 

+ Model Performance

In [51]:
########### Write Your Code Here ###########
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    y_test=[]
    y_predicted_cls=[]
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_test.extend(labels.numpy())
        y_predicted_cls.extend(predicted.numpy())

    print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')
############################################

Accuracy of the model on the test images: 96.97%


In [52]:

y_test = np.array(y_test)
y_predicted_cls = np.array(y_predicted_cls)

#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted_cls))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98       980
           1       0.99      0.99      0.99      1135
           2       0.96      0.97      0.97      1032
           3       0.96      0.97      0.97      1010
           4       0.98      0.96      0.97       982
           5       0.98      0.95      0.96       892
           6       0.96      0.97      0.97       958
           7       0.98      0.96      0.97      1028
           8       0.95      0.97      0.96       974
           9       0.96      0.97      0.97      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000

